In [1]:
from utils.dataframes import *
import pandas as pd
import numpy as np

# Challenge: It seems like CMP still charges for delivery and taxes in cases where the electricity is fully supplied by Ampion. Therefore, only `kwh_delivered` from CMP is immediately useful for the waterfall, and a different strategy for `service_charge` and `taxes` needs to be employed based on the timeframes of the bills (e.g. across this bill's dates, you paid this much)

explode = {s: df.rename(columns = {'id': 'dim_bills_id'}) 
              for s, df in dim_bills.explode('billing_interval')
                                    .assign(date = lambda x: pd.to_datetime(x['billing_interval']))
                                    .groupby('source')}

# Step 2: Merge with meter usage and dimension tables
flat_df = meter_usage.assign(timestamp = lambda df: pd.to_datetime(df['interval_end_datetime'], format = '%m/%d/%Y %I:%M:%S %p')) \
                     .merge(dim_datetimes,     on = 'timestamp', how = 'left', suffixes = ('', '_dat')) \
                     .merge(dim_meters,        on = 'meter_id',  how = 'left', suffixes = ('', '_met')) \
                     .reset_index(drop = True)

bill_fields = ['account_number', 'date']
matched_c = flat_df.merge(explode['CMP'],    on = bill_fields, how = 'inner')
matched_a = flat_df.merge(explode['Ampion'], on = bill_fields, how = 'inner')

# int_df = pd.concat([matched_c, matched_a]) \
#            .apply(lambda col: col.fillna(0) if col.dtype.kind in 'biufc' else col) \
#            .drop_duplicates()

# # Step 3: Calculate total kWh recorded
# int_df['total_recorded_kwh'] = int_df.groupby(['invoice_number', 'kwh_delivered'])['kwh'].transform('sum')
# int_df['kwh_ratio']          = int_df['kwh'] / int_df['total_recorded_kwh']

In [2]:
# # Sort the DataFrame
# sort_df = int_df.sort_values(by = ['source', 'invoice_number', 'timestamp'])
# sort_df.reset_index(drop = True, inplace = True)
                
# group_df = sort_df.groupby(['source', 'invoice_number', 'kwh_delivered'])

# # Initialize new columns
# sort_df['kwh_left'] = 0.0
# sort_df['kwh_used'] = 0.0

# # Calculate for CMP
# c_mask = sort_df['source'] == 'CMP'
# sort_df.loc[c_mask, 'kwh_left'] = (group_df['kwh_delivered'].transform('first') - 
#                                    group_df['kwh'].cumsum().iloc[::-1]).clip(lower = 0)
# sort_df.loc[c_mask, 'kwh_used'] = np.minimum(sort_df['kwh'], sort_df['kwh_left'])

# # Calculate kwh_remaining for CMP
# sort_df['kwh_remaining'] = sort_df['kwh'] - sort_df['kwh_used']

# # Update kwh_remaining in int_df using the map
# int_df['kwh_remaining'] = sort_df['kwh_remaining']

# # Calculate for Ampion
# a_mask = sort_df['source'] == 'Ampion'
# sort_df.loc[a_mask, 'kwh_left'] = (group_df['kwh_delivered'].transform('first') - 
#                                    group_df['kwh_remaining'].cumsum()).clip(lower = 0)
# sort_df.loc[a_mask, 'kwh_used'] = np.minimum(sort_df['kwh_remaining'], sort_df['kwh_left'])

# # Final cleanup
# int_df['kwh_used'] = sort_df['kwh_used']
# int_df['kwh_used'] = int_df['kwh_used'].fillna(0).astype(float)

# int_df

In [4]:
# Step 1: Waterfall for CMP (matched_c)
# Group by necessary fields
group_c = matched_c.groupby(['source', 'invoice_number', 'kwh_delivered'])

# Initialize columns
matched_c['kwh_left'] = 0.0
matched_c['kwh_used'] = 0.0

# Calculate 'kwh_left' and 'kwh_used' for CMP
matched_c['kwh_left']   = (group_c['kwh_delivered'].transform('first') - 
                           group_c['kwh'].cumsum()).clip(lower=0)
matched_c['kwh_used']   = np.minimum(matched_c['kwh'], matched_c['kwh_left'])
matched_c['kwh_unused'] = matched_c['kwh'] - matched_c['kwh_used']

# Step 2: Waterfall for Ampion (matched_a) using 'kwh_remaining' from CMP
# Group by necessary fields
group_a = matched_a.groupby(['source', 'invoice_number', 'kwh_delivered'])

# Initialize columns
matched_a['kwh_left'] = 0.0
matched_a['kwh_used'] = 0.0
matched_a['kwh']      = matched_c['kwh_unused']

# Calculate 'kwh_left' and 'kwh_used' for Ampion
matched_a['kwh_left']   = (group_a['kwh_delivered'].transform('first') - 
                           group_a['kwh'].cumsum()).clip(lower=0)
matched_a['kwh_used']   = np.minimum(matched_a['kwh'], matched_a['kwh_left'])
matched_a['kwh_unused'] = matched_a['kwh'] - matched_a['kwh_used']

# Combine the results
int_df = pd.concat([matched_c, matched_a])

int_df


,service_point_id,meter_id,interval_end_datetime,meter_channel,kwh,account_number,timestamp,id,increment,hour,...,kwh_delivered,service_charge,taxes,delivery_rate,supply_rate,source,billing_interval,kwh_left,kwh_used,kwh_unused
0,2300822246,L108605388,10/1/2022 12:00:00 AM,10,0.594,30010320353,2022-10-01 00:00:00,69401,0,0,...,0,25.67,1.41,NaN,NaN,CMP,2022-10-01,0.000,0.000,0.594
1,2300822246,L108605388,10/1/2022 12:15:00 AM,10,0.101,30010320353,2022-10-01 00:15:00,69402,15,0,...,0,25.67,1.41,NaN,NaN,CMP,2022-10-01,0.000,0.000,0.101
2,2300822246,L108605388,10/1/2022 12:30:00 AM,10,0.104,30010320353,2022-10-01 00:30:00,69403,30,0,...,0,25.67,1.41,NaN,NaN,CMP,2022-10-01,0.000,0.000,0.104
3,2300822246,L108605388,10/1/2022 12:45:00 AM,10,0.106,30010320353,2022-10-01 00:45:00,69404,45,0,...,0,25.67,1.41,NaN,NaN,CMP,2022-10-01,0.000,0.000,0.106
4,2300822246,L108605388,10/1/2022 1:00:00 AM,10,0.099,30010320353,2022-10-01 01:00:00,69405,0,1,...,0,25.67,1.41,NaN,NaN,CMP,2022-10-01,0.000,0.000,0.099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157338,2300588897,L108607371,9/30/2022 7:00:00 PM,10,0.189,35012790198,2022-09-30 19:00:00,69381,0,19,...,1250,0.00,0.00,0.0,0.166968,Ampion,2022-09-30,922.937,0.189,0.000
157339,2300588897,L108607371,9/30/2022 8:00:00 PM,10,0.247,35012790198,2022-09-30 20:00:00,69385,0,20,...,1250,0.00,0.00,0.0,0.166968,Ampion,2022-09-30,922.690,0.247,0.000
157340,2300588897,L108607371,9/30/2022 9:00:00 PM,10,0.284,35012790198,2022-09-30 21:00:00,69389,0,21,...,1250,0.00,0.00,0.0,0.166968,Ampion,2022-09-30,922.406,0.284,0.000
157341,2300588897,L108607371,9/30/2022 10:00:00 PM,10,0.151,35012790198,2022-09-30 22:00:00,69393,0,22,...,1250,0.00,0.00,0.0,0.166968,Ampion,2022-09-30,922.255,0.151,0.000


In [16]:
sources = ['CMP']

print(dim_bills[dim_bills['source'].isin(sources)]['kwh_delivered'].sum(),
int_df[int_df['source'].isin(sources)]['kwh_used'].sum())

191761 190512.645


Allocation seems to be working great for CMP and its range of values. However, when Ampion is incorporated, things start getting seriously messed up.

- Simplify the approach (just allocate from the beginning to the end for both)
- Even if the timeframes are different, if `cmp` is done first, we can replace `matched_a['kwh']` with `kwh_remaining` and continue the window

For future curation, if `kwh_used` is 0, use original `flat_df['kwh']`

In [ ]:
df_test = flat_df.groupby('month_start').agg({'kwh': 'sum'}) \
             .merge(int_df.groupby('month_start').agg({'kwh_used': 'sum'}), on = 'month_start')

print(flat_df['kwh'].sum(), int_df['kwh_used'].sum())
df_test

In [ ]:
df = flat_df

df[df.index == 355016]

# df[(df['month_start'] == '2020-10-01') & (df['account_number'] == '30010894035')]

# unmatched: 335567
# flat: 355016

In [ ]:
# # Step 6: Compute cost metrics
# df = pd.DataFrame(index = int_df.index)
# df['dim_datetimes_id']  = int_df['id']
# df['dim_meters_id']     = int_df['id_met']
# df['dim_bills_id']      = int_df['dim_bills_id']
# df['account_number']    = int_df['account_number']
# df['kwh']               = int_df['kwh']
# df['delivery_cost']     = int_df['kwh_used']       * int_df['delivery_rate']
# df['service_cost']      = int_df['service_charge'] * int_df['kwh_ratio']
# df['supply_cost']       = int_df['kwh_used']       * int_df['supply_rate']
# df['tax_cost']          = int_df['taxes']          * int_df['kwh_ratio']
# df['total_cost']        = df.filter(regex = '_cost$').sum(axis = 1)